In [ ]:
!pip install selenium
!apt-get update
!apt install chromium-chromedriver

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 63.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 499.2/499.2 kB 32.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.8/129.8 kB 11.5 MB/s eta 0:00:00
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.4.0
    Uninstalling urllib3-2.4.0:
      Successfully uninstalled urllib3-2.4.0
Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Get:7 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 h

In [5]:
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from pprint import pprint
import time


options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/110.0.5481.178 Safari/537.36")

driver = webdriver.Chrome(options=options)

# === Load the site ===
url = "https://www.inmotionhosting.com/"
driver.get(url)
# time.sleep(10)
wait = WebDriverWait(driver, 5)


# url = "https://www.inmotionhosting.com/shared-hosting"
# driver.get(url)
# # time.sleep(10)
# wait = WebDriverWait(driver, 5)

# print(driver.page_source)



def load_tariff_detail(link):
  driver.get(link)
  wait = WebDriverWait(driver, 10)

  container = driver.find_element(By.CLASS_NAME, 'imh-product-rostrum')
  switch_buttons_container = container.find_elements(By.CLASS_NAME, 'imh-term-selector-container')[-1]
  switch_buttons = switch_buttons_container.find_elements(By.TAG_NAME, 'button')

  cards = driver.find_element(By.CLASS_NAME, 'imh-rostrum-container') \
          .find_elements(By.CLASS_NAME, "imh-rostrum-card")

  cards_data = []

  for card in cards:
    title_element = card.find_element(By.CLASS_NAME, 'imh-rostrum-card-title')
    title = title_element.get_attribute('innerHTML').strip()

    tariff_prices = []
    for index, button in enumerate(switch_buttons):
      driver.execute_script(f"document.getElementsByClassName('imh-term-selector')[{index}].click()")

      duration = button.get_attribute('innerHTML').strip()

      try:
        price_element = card.find_element(By.CSS_SELECTOR,'.imh-switcher.active') \
                        .find_element(By.CLASS_NAME, 'rostrum-price')
        price = price_element.get_attribute('innerHTML').strip()
        tariff_prices.append({
            'duration': duration,
            'price': price
        })

      except:
        tariff_prices.append({
            'duration': duration,
            'price': 'Not found'
        })

    cards_data.append({
        'title': title,
        'prices': tariff_prices
    })
    # print(card.get_attribute('innerHTML'))
    # print()

  return cards_data


def load_main_cards():
  cards = driver.find_elements(By.CLASS_NAME, "imh-rostrum-card")

  cards = cards[:4]
  cards_data = []

  for card in cards:
    title_element = card.find_element(By.CLASS_NAME, 'imh-rostrum-card-title')
    title = title_element.get_attribute('innerHTML').strip()

    price_element = card.find_element(By.CLASS_NAME, 'rostrum-price')
    price = price_element.get_attribute('innerHTML').strip()

    link_element = card.find_element(By.CLASS_NAME, 'cta-link')
    link = link_element.get_attribute('href')

    cards_data.append({
        'title': title,
        'price': price,
        'link': link,
    })

  for card_data in cards_data:
    sub_tariffs = load_tariff_detail(card_data['link'])

    card_data['details'] = sub_tariffs

  return cards_data


result = load_main_cards()
pprint(result)

driver.quit()

[{'details': [{'prices': [{'duration': '3 Year', 'price': '$3.19'},
                          {'duration': '2 Year', 'price': '$3.19'},
                          {'duration': '1 Year', 'price': '$2.99'},
                          {'duration': '1 Month', 'price': '$13.49'}],
               'title': 'Core'},
              {'prices': [{'duration': '3 Year', 'price': '$4.99'},
                          {'duration': '2 Year', 'price': '$4.99'},
                          {'duration': '1 Year', 'price': '$4.79'},
                          {'duration': '1 Month', 'price': '$15.99'}],
               'title': 'Launch'},
              {'prices': [{'duration': '3 Year', 'price': '$4.99'},
                          {'duration': '2 Year', 'price': '$4.99'},
                          {'duration': '1 Year', 'price': '$4.79'},
                          {'duration': '1 Month', 'price': '$15.99'}],
               'title': 'Power'},
              {'prices': [{'duration': '3 Year', 'price': '$10.99'},
    